# Hospital Data Exploration

### In this task, I will be working with simulated hospital data based on admissions. The csv files admission_surg and admission_med contain information of patients’ admissions to a hospital’s surgical and medical departments respectively. Imaging.csv contains the medical imaging data results collected throughout each admission. The column ID found in all three files identifies each unique admission.

##### Import the required libraries

In [1]:
import pandas as pd
import numpy as np

##### Load the data

In [2]:
df_surg=pd.read_csv('C:\\Users\\dell\\Downloads\\admissions_surg.csv')
df_med=pd.read_csv('C:\\Users\\dell\\Downloads\\admissions_med.csv')
df_img=pd.read_csv("C:\\Users\\dell\\Downloads\\imaging.csv")

##### De-identify the confidential ID's

In [3]:
med_id=df_med['ID']
surg_ID=df_surg['ID']

In [4]:
comb_ID=pd.concat([med_id, surg_ID]).drop_duplicates().reset_index(drop=True)
comb_ID

0        1064
1        1066
2        1074
3        1108
4        1137
        ...  
3495    99787
3496    99791
3497    99855
3498    99858
3499    99905
Name: ID, Length: 3500, dtype: int64

In [5]:
q1b=pd.DataFrame({'ID': comb_ID})
q1b

,ID
0,1064
1,1066
2,1074
3,1108
4,1137
...,...
3495,99787
3496,99791
3497,99855
3498,99858


In [6]:
q1b['DE_ID']= range(1, len(q1b)+1)
q1b

,ID,DE_ID
0,1064,1
1,1066,2
2,1074,3
3,1108,4
4,1137,5
...,...,...
3495,99787,3496
3496,99791,3497
3497,99855,3498
3498,99858,3499


In [7]:
q1b['DE_ID'].is_unique

True

##### Place back the De-identified ID's into original Datasets and remove confidential ID's

In [8]:
def new_datasets(dataset):
   return dataset.merge(q1b, on='ID').drop(columns='ID')

In [9]:
df_med_ID= new_datasets(df_med)
df_surg_ID= new_datasets(df_surg)
df_img_ID=new_datasets(df_img)

#### Combine surgical and medical csv's, and merge with imaging data using DE_ID

##### Before combining check if the column names are same or not, to combine vertically
##### If not, make it same before combining 

In [11]:
df_med_ID.columns = df_med_ID.columns.str.lower()
df_surg_ID.columns = df_surg_ID.columns.str.lower()
df_img_ID.columns = df_img_ID.columns.str.lower()

In [12]:
df_surg_ID.columns = df_surg_ID.columns.str.replace('.', '_')

In [13]:
df_med_ID.columns


Index(['admission_date', 'admission_time', 'discharge_date', 'discharge_time',
       'department', 'gender', 'age', 'main_diagnosis_icd10',
       'main_diagnosis_name', 'de_id'],
      dtype='object')

In [14]:
df_surg_ID.columns

Index(['admission_date', 'admission_time', 'discharge_date', 'discharge_time',
       'department', 'gender', 'age', 'main_diagnosis_icd10',
       'main_diagnosis_name', 'de_id'],
      dtype='object')

### Merge: When there is key column ( 1 or 2) that matches
### Concat: When there is no matching column (axis=1, keeps all columns (horizontal); axis=0, keeps all rows (vertical))

In [15]:
admissions_img= pd.concat([df_med_ID, df_surg_ID], axis=0)
admissions_img.head(2)

,admission_date,admission_time,discharge_date,discharge_time,department,gender,age,main_diagnosis_icd10,main_diagnosis_name,de_id
0,1985-10-19,04:27,1986-01-01,NaN,General Internal Medicine,M,63.0,N830,Ovarian cyst,1
1,1990-06-26,21:06,1990-09-03,21:03,General Internal Medicine,F,63.0,N410,Inflammatory conditions of male genital organs,2


In [16]:
final_merged= admissions_img.merge(df_img_ID, on='de_id')
final_merged.head(2)

,admission_date,admission_time,discharge_date,discharge_time,department,gender,age,main_diagnosis_icd10,main_diagnosis_name,de_id,test_name,ordered_date_time,performed_date,performed_time,technician_name,brief_report
0,1985-10-19,04:27,1986-01-01,NaN,General Internal Medicine,M,63.0,N830,Ovarian cyst,1,US,NaN,1985-12-17,10:27,Trevon Hopson,No significant abnormality
1,1985-10-19,04:27,1986-01-01,NaN,General Internal Medicine,M,63.0,N830,Ovarian cyst,1,US PELVIS,NaN,1985-12-02,11:40,Claire Melko,Indication: normal


In [17]:
df_surg_ID.head(2)

,admission_date,admission_time,discharge_date,discharge_time,department,gender,age,main_diagnosis_icd10,main_diagnosis_name,de_id
0,1998-02-01,07:02,1998-03-31,07:17,General Surgery,M,NaN,E0800,Diabetes mellitus with complications,2306
1,2010-11-30,04:33,2011-03-25,20:59,General Surgery,F,24.0,M0500,Rheumatoid arthritis and related disease,2307


##### Create new column called length_of_stay variable defined as discharge date and time minus admission date and time (in days)

In [19]:
final_merged['discharge_time'].fillna('00:00', inplace=True)
final_merged['admission_time'].fillna('00:00', inplace=True)

In [20]:
final_merged.dtypes

admission_date           object
admission_time           object
discharge_date           object
discharge_time           object
department               object
gender                   object
age                     float64
main_diagnosis_icd10     object
main_diagnosis_name      object
de_id                     int64
test_name                object
ordered_date_time        object
performed_date           object
performed_time           object
technician_name          object
brief_report             object
dtype: object

In [21]:
final_merged['admission_date_time']=pd.to_datetime(final_merged['admission_date']+ ' ' +final_merged['admission_time'], format='%Y-%m-%d %H:%M')

In [22]:
final_merged['discharge_date_time'] = pd.to_datetime(final_merged['discharge_date']+ ' ' +final_merged['discharge_time'], format='%Y-%m-%d %H:%M')
final_merged['discharge_date_time']

0      1986-01-01 00:00:00
1      1986-01-01 00:00:00
2      1990-09-03 21:03:00
3      1994-12-22 07:21:00
4      2005-04-19 00:00:00
               ...        
4995   2006-08-21 02:45:00
4996   1999-08-26 23:55:00
4997   1999-08-26 23:55:00
4998   1999-08-26 23:55:00
4999   2010-05-30 14:18:00
Name: discharge_date_time, Length: 5000, dtype: datetime64[ns]

In [23]:
final_merged['length_of_stay']=(final_merged['discharge_date_time']-final_merged['admission_date_time']).dt.days

In [24]:
final_merged.head(2)

,admission_date,admission_time,discharge_date,discharge_time,department,gender,age,main_diagnosis_icd10,main_diagnosis_name,de_id,test_name,ordered_date_time,performed_date,performed_time,technician_name,brief_report,admission_date_time,discharge_date_time,length_of_stay
0,1985-10-19,04:27,1986-01-01,00:00,General Internal Medicine,M,63.0,N830,Ovarian cyst,1,US,NaN,1985-12-17,10:27,Trevon Hopson,No significant abnormality,1985-10-19 04:27:00,1986-01-01,73
1,1985-10-19,04:27,1986-01-01,00:00,General Internal Medicine,M,63.0,N830,Ovarian cyst,1,US PELVIS,NaN,1985-12-02,11:40,Claire Melko,Indication: normal,1985-10-19 04:27:00,1986-01-01,73


##### Calculate the mean length_of_stay for each department.

In [132]:
final_merged.groupby('department')['length_of_stay'].mean()

department
Addiction Services            99.980983
General Internal Medicine     97.745682
General Surgery              100.395425
Obstetrics                   104.267894
Oncology                     102.308989
Palliative Care               95.750943
Name: length_of_stay, dtype: float64

In [26]:
df_img_ID.head(2)

,test_name,ordered_date_time,performed_date,performed_time,technician_name,brief_report,de_id
0,US,NaN,1985-12-17,10:27,Trevon Hopson,No significant abnormality,1
1,US PELVIS,NaN,1985-12-02,11:40,Claire Melko,Indication: normal,1


#### Filter the first performed test for each test_name and save the resulting dataframe as q4_df

In [30]:
df_img_ID['performed_date_time']= pd.to_datetime(df_img_ID['performed_date']+ ' ' + df_img_ID['performed_time'], format='%Y-%m-%d %H:%M')

In [133]:
earliest_tests = df_img_ID.loc[df_img_ID.groupby('de_id')['performed_date_time'].idxmin()]

# 2. Create q4_df with de_id, test_name, and performed_date_time
q4_df = earliest_tests[['test_name', 'performed_date_time']].reset_index(drop=True)
q4_df


,test_name,performed_date_time
0,US PELVIS,1985-12-02 11:40:00
1,Abdomen CT,1990-08-05 12:26:00
2,CT neck + head,1994-11-05 01:36:00
3,Ultrasound,2005-04-04 15:24:00
4,CT,1997-08-04 15:51:00
...,...,...
3089,US,2004-07-21 06:01:00
3090,CT,1996-09-07 23:35:00
3091,CT,2006-05-24 19:13:00
3092,Abdomen CT,1999-08-07 03:12:00


##### Remove any rows with missing values (NA) in 2 or more variables and name the resulting data frame q5_df. Report the missing rate (%) in each field from this data frame.

In [36]:
q5_df = final_merged.dropna(thresh=len(admissions_img.columns) - 2)

In [37]:
missing_rate = (q5_df.isnull().mean() * 100).round(2)
missing_rate

admission_date           0.00
admission_time           0.00
discharge_date           0.00
discharge_time           0.00
department               0.00
gender                   0.00
age                      5.98
main_diagnosis_icd10     0.00
main_diagnosis_name      8.18
de_id                    0.00
test_name                0.00
ordered_date_time       33.96
performed_date           0.00
performed_time           0.00
technician_name          0.00
brief_report             0.00
admission_date_time      0.00
discharge_date_time      0.00
length_of_stay           0.00
dtype: float64

##### Using the imaging data, create a frequency table of unique test_name, with decreasing order of frequency. Name this table img_mapper.

In [39]:
img_mapper= df_img_ID.groupby('test_name')['test_name'].count().sort_values(ascending=False)

In [82]:
img_mapper_df = img_mapper.reset_index(name='frequency')
img_mapper_df= pd.DataFrame({
    'test_name': img_mapper_df['test_name'],
    'frequency': img_mapper_df['frequency']
})

#### In img_mapper, add a column named mapped_test which classifies each unique test_name into 3 categories: "US" (or Ultrasound), "CT" and "Others"

In [84]:
img_mapper_df.loc[img_mapper_df['test_name'].str.contains('CT', case=False), 'mapped_test'] = 'CT'


In [90]:
img_mapper_df.loc[img_mapper_df['test_name'].str.contains('US|Ultrasound', case=False), 'mapped_test'] = 'US'

In [92]:
img_mapper_df.loc[~img_mapper_df['test_name'].str.contains('US|Ultrasound|CT', case=False), 'mapped_test'] = 'Others'

In [93]:
img_mapper_df

,test_name,frequency,mapped_test
0,US,680,US
1,CT,511,CT
2,Ultrasound,441,US
3,ct neck and head,408,CT
4,US ABDOMEN,396,US
5,CT neck + head,385,CT
6,ABDOMEN/PELVIS US,313,US
7,CT - ABDOMEN,310,CT
8,NECK AND HEAD CT,248,CT
9,Doppler Ultrasound,233,US


#### In img_mapper, add another column named mapped_bodypart, which classifies each unique test_name into 3 categories, "Neck/Head", "Abdomen/pelvis", "Others". Display the table head.

In [94]:
img_mapper_df.loc[img_mapper_df['test_name'].str.contains('Neck|Head', case=False), 'mapped_bodypart'] = 'Neck/Head'
img_mapper_df.loc[img_mapper_df['test_name'].str.contains('Abdomen|pelvis', case=False), 'mapped_bodypart'] = 'Abdomen/pelvis'
img_mapper_df.loc[~img_mapper_df['test_name'].str.contains('Abdomen|pelvis|Neck|Head', case=False), 'mapped_bodypart'] = 'Others'

In [95]:
img_mapper_df

,test_name,frequency,mapped_test,mapped_bodypart
0,US,680,US,Others
1,CT,511,CT,Others
2,Ultrasound,441,US,Others
3,ct neck and head,408,CT,Neck/Head
4,US ABDOMEN,396,US,Abdomen/pelvis
5,CT neck + head,385,CT,Neck/Head
6,ABDOMEN/PELVIS US,313,US,Abdomen/pelvis
7,CT - ABDOMEN,310,CT,Abdomen/pelvis
8,NECK AND HEAD CT,248,CT,Neck/Head
9,Doppler Ultrasound,233,US,Others


#### Investigate possible data quality issues in other source .csv file

In [101]:
Q7_admdad = pd.read_csv("C:\\Users\\dell\\Downloads\\Q7_admdad.csv")

In [102]:
Q7_admdad 

,encounter_id,age,country,birth_date,admission_date,discharge_date,hospital_num
0,704339,90,Canada,6/15/1933,45081,6/18/2023,101.0
1,508188,99,Canada,5/16/1924,45126,8/8/2023,101.0
2,365079,27,Canada,8/9/1996,45117,7/31/2023,101.0
3,881762,7,Canada,10/21/2015,1/15/2023,1/22/2023,102.0
4,775894,42,Canada,1/16/1981,6/29/2023,7/15/2023,102.0
...,...,...,...,...,...,...,...
166,873422,59,Canada,3/16/1965,12/13/2023,12/25/2023,102.0
167,612558,10,Canada,12/16/2013,10/30/2023,11/13/2023,101.0
168,163280,77,Canada,12/16/1972,8/24/2023,9/7/2023,102.0
169,208162,96,Canada,NaN,NaN,NaN,NaN


In [ ]:
#admission date contains different values like "45081" 

In [103]:
Q7_admdad['admission_date'].unique()

array(['45081', '45126', '45117', '1/15/2023', '6/29/2023', '9/16/2023',
       '44977', '12/11/2023', '12/20/2023', '45156', '3/8/2023',
       '6/10/2023', '5/5/2023', '1/21/2023', '7/8/2023', '45155',
       '11/15/2023', '11/1/2023', '12/24/2023', '4/2/2023', '11/9/2023',
       '12/25/2023', '5/4/2023', '8/23/2023', '6/5/2023', '44987',
       '45100', '7/14/2023', '10/11/2023', '10/10/2023', '45017',
       '10/5/2023', '8/20/2023', '11/26/2023', '6/30/2023', '5/18/2023',
       '2/19/2023', '1/8/2023', '1/31/2023', '1/16/2023', '12/28/2023',
       '45154', '5/6/2023', '10/17/2023', '6/26/2023', '2/9/2023',
       '4/4/2023', '4/16/2023', '45137', '11/14/2023', '7/6/2023',
       '8/6/2023', '10/30/2023', '4/17/2023', '4/26/2023', '6/19/2023',
       '7/16/2023', '44948', '5/11/2023', '6/14/2023', '3/12/2023',
       '2/13/2023', '45010', '45269', '2/27/2023', '9/18/2023',
       '8/9/2023', '3/30/2023', '7/18/2023', '8/18/2023', '5/22/2023',
       '44941', '44929', '11/8/2023'

In [104]:
Q7_admdad.isnull().sum()

encounter_id      0
age               0
country           0
birth_date        1
admission_date    2
discharge_date    2
hospital_num      2
dtype: int64

In [ ]:
#country contains different values like date

In [105]:
Q7_admdad['country'].unique()

array(['Canada', 'United States', '3/16/2023'], dtype=object)

In [ ]:
#encounter_id contains different values like date

In [106]:
Q7_admdad['encounter_id'].unique()

array(['704339', '508188', '365079', '881762', '775894', '811026',
       '968137', '618965', '200580', '537062', '985237', '750030',
       '635899', '339087', '553671', '684213', '243160', '573685',
       '227726', '746354', '357897', '606544', '857482', '210072',
       '267002', '154036', '853268', '871535', '363166', '754196',
       '406728', '765847', '706824', '763228', '463208', '301156',
       '427081', '801143', '578566', '757666', '378889', '561221',
       '633241', '860732', '771243', '294441', '335407', '415480',
       '146208', '230055', '201378', '880182', '667296', '956430',
       '468221', '453376', '539783', '881843', '449649', '684435',
       '724448', '780134', '778849', '705495', '782558', '846235',
       '180870', '282750', '790497', '485313', '771181', '942845',
       '685215', '759189', '104040', '998905', '731476', '723015',
       '106226', '974322', '578364', '638517', '927092', '881361',
       '165354', '921232', '123848', '436910', '734167', '3005

In [ ]:
#age is not consistent, and also has date in few rows 

In [107]:
Q7_admdad['age'].unique()

array(['90', '99', '27', '7', '42', 'Age: 65', '43', '38', '100',
       'Age: 50', '55', 'Age: 77', '10', '62', '41', '35', '67', '73',
       '68', '60', '22', '74', '75', '8', '63', '51', '77', 'Age: 26',
       'Age: 58', '39', '95', '26', 'Age: 96', 'Age: 94', '82', '1', '71',
       '94', '89', '18', '49', 'Age: 61', '25', 'Age: 92', '32', '3', '5',
       'Age: 66', '23', '86', '56', 'Age: 60', '98', '97', '13', '34',
       '14', 'Age: 25', '11', '81', '79', '33', '40', '70', '69', '85',
       '96', 'Age: 90', 'Age: 24', 'Age: 52', '52', '88', '44', '93',
       'Age: 79', '65', '12', 'Age: 44', '50', '58', 'Age: 99', 'Age: 76',
       '78', '47', 'Age: 32', '92', 'Age: 20', '45', '59', '3/3/2023'],
      dtype=object)

In [ ]:
#birth date contains different values like "101"

In [108]:
Q7_admdad['birth_date'].unique()

array(['6/15/1933', '5/16/1924', '8/9/1996', '10/21/2015', '1/16/1981',
       '2/1/2001', '1/22/1980', '11/17/1985', '2/28/1924', '10/5/1929',
       '4/17/1935', '10/2/1958', '9/6/1946', '6/20/2013', '4/5/1961',
       '4/1/1982', '5/3/1974', '3/22/1989', '9/6/1956', '12/28/1949',
       '7/17/1927', '12/29/1963', '12/31/2000', '11/18/1949', '4/9/1948',
       '6/11/2015', '1/6/1960', '5/5/1975', '7/7/1946', '5/9/1997',
       '11/27/1965', '6/5/1984', '3/15/1929', '6/27/1997', '5/5/1927',
       '1/17/1925', '10/25/1961', '12/22/1928', '5/23/1946', '8/3/1949',
       '12/6/1940', '6/10/1933', '11/2/2022', '11/13/1974', '6/27/1929',
       '8/4/1934', '6/10/2005', '6/22/1941', '5/31/1974', '8/13/1962',
       '10/21/1998', '9/2/2015', '5/23/1931', '1/20/1992', '8/19/1973',
       '3/8/2020', '4/29/2018', '8/25/1957', '2/25/2000', '5/20/1937',
       '12/21/1966', '4/12/1960', '11/20/1963', '12/3/1924', '3/8/1926',
       '8/18/1923', '10/11/2010', '1/22/1988', '2/20/1983', '6/6/1989'

In [ ]:
#encounter_id should be unique, but has 20 duplicated values

In [117]:
Q7_admdad[Q7_admdad['encounter_id'].duplicated()]

,encounter_id,age,country,birth_date,admission_date,discharge_date,hospital_num
56,243160,Age: 50,Canada,5/3/1974,11/15/2023,11/29/2023,115.0
57,811026,Age: 65,Canada,2/1/2001,9/16/2023,10/5/2023,102.0
67,811026,Age: 65,Canada,2/1/2001,9/16/2023,10/5/2023,102.0
71,357897,68,Canada,7/17/1927,11/9/2023,11/13/2023,101.0
83,468221,26,United States,8/19/1973,10/30/2023,11/20/2023,120.0
85,468221,26,United States,8/19/1973,10/30/2023,11/20/2023,120.0
86,750030,55,Canada,10/2/1958,6/10/2023,6/23/2023,101.0
98,468221,26,United States,8/19/1973,10/30/2023,11/20/2023,120.0
103,811026,Age: 65,Canada,2/1/2001,9/16/2023,10/5/2023,102.0
106,750030,55,Canada,10/2/1958,6/10/2023,6/23/2023,101.0
